# 파트 02
----
아파트 매매 데이터에서 불필요한 칼럼들을 제거한 뒤 추후 인구이동 데이터와 비교분석을 위해 주소 간략화 및 행자부코드를 부여한다.

행자부코드 목록에 명기된 주소보다 아파트 매매 데이터에는 더 상세히 기술되어 있어 주소 문자열을 처음 두 단어 기준으로 간략화 시켰다. 따라서 일반적으로 대도시인 특별 행정구역의 경우 시+구의 단위로 쓰이게 되며 그 외에는 도+시 단위로 쓰이게 된다.

세종특별시의 경우 생긴지 얼마 안된 이유로 잦은 행정구역의 변화가 있었기에 임의로 이전 행정구역을 사용하도록 수정을 한다.

마지막으로 건축년도가 null값으로 되어있는 데이터 역시 수정을 가해 null값이 전혀 없는 깔끔한 데이터셋을 만든다.

In [2]:
%matplotlib inline
import re

In [3]:
path = 'raw_data\\'

In [4]:
df_estate = pd.read_csv(path + 'real_estate.csv', index_col=0, encoding='utf-8')

In [5]:
df_address_code = pd.read_excel(path + 'address_code.xlsx', header=2)

In [6]:
# checking progress
df_estate.tail()

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
5863875,충청북도 충주시 호암동,547-6,547,6,호반현대,59.76,201512,21~31,"10,500",8,1994.0,신촌2길
5863876,충청북도 충주시 호암동,547-6,547,6,호반현대,59.76,201512,21~31,"10,200",10,1994.0,신촌2길
5863877,충청북도 충주시 호암동,905,905,0,호암동수채,84.93,201512,11~20,"19,700",12,2007.0,호암중앙1로
5863878,충청북도 충주시 호암동,221-23,221,23,호암리버빌(1단지),84.68,201512,11~20,"15,900",9,2002.0,원호암5길
5863879,충청북도 충주시 호암동,221-1,221,1,호암리버빌(2단지),84.68,201512,11~20,"16,000",10,2002.0,호암중앙2로


In [7]:
# remove unnecessary columns
del df_estate[u'번지']
del df_estate[u'본번']
del df_estate[u'부번']
del df_estate[u'계약일']
del df_estate[u'도로명']
del df_estate[u'단지명']

In [8]:
# see how the data frame has changed
df_estate.tail()

,시군구,전용면적(㎡),계약년월,거래금액(만원),층,건축년도
5863875,충청북도 충주시 호암동,59.76,201512,"10,500",8,1994.0
5863876,충청북도 충주시 호암동,59.76,201512,"10,200",10,1994.0
5863877,충청북도 충주시 호암동,84.93,201512,"19,700",12,2007.0
5863878,충청북도 충주시 호암동,84.68,201512,"15,900",9,2002.0
5863879,충청북도 충주시 호암동,84.68,201512,"16,000",10,2002.0


In [9]:
# function to shrink multiple white spaces into a single space
def shrink_spaces(address):
    return re.sub(u'\s+', u' ', address)

# function that returns the first two words of the string
subaddress_regex = re.compile(u'\S+\s+\S+')
def shrink_address(address):
    return address[:subaddress_regex.search(address).end()]

In [10]:
# remove whitespaces at the beginning of strings and at the end of strings
df_estate[u'시군구'] = df_estate[u'시군구'].str.strip()

# shrink multiple white spaces
df_estate[u'시군구'] = df_estate[u'시군구'].apply(shrink_spaces)

# cut off addresses after the first two words
df_estate[u'시군구'] = df_estate[u'시군구'].apply(shrink_address)

In [11]:
# similar process as above
df_address_code[u'행정구역명'] = df_address_code[u'행정구역명'].apply(shrink_spaces)

In [12]:
# creating a series to map address strings to numeric codes
sr_address_code = pd.Series(data=df_address_code.set_index(u'행정구역명')[u'행자부코드'], copy=True)

# remove the first occurring duplicates
# here we assume the later occurring duplicates correspond to the updated address codes (I'm not 100% sure)
sr_address_code = sr_address_code[~sr_address_code.index.duplicated(keep='first')]

In [13]:
# just checking
sr_address_code.tail()

행정구역명
제주특별자치도 서귀포시 서홍동    5013058000
제주특별자치도 서귀포시 대륜동    5013059000
제주특별자치도 서귀포시 대천동    5013060000
제주특별자치도 서귀포시 중문동    5013061000
제주특별자치도 서귀포시 예래동    5013062000
Name: 행자부코드, dtype: int64

In [14]:
# now create a column named address code
df_estate[u'행자부코드'] = df_estate[u'시군구'].map(sr_address_code)

In [15]:
# we see below address codes are given float values instead of integer values
# this probably means there are some addresses that did not get mapped properly
# hence getting assigned nan (float) values
df_estate.tail()

,시군구,전용면적(㎡),계약년월,거래금액(만원),층,건축년도,행자부코드
5863875,충청북도 충주시,59.76,201512,"10,500",8,1994.0,4.313000e+09
5863876,충청북도 충주시,59.76,201512,"10,200",10,1994.0,4.313000e+09
5863877,충청북도 충주시,84.93,201512,"19,700",12,2007.0,4.313000e+09
5863878,충청북도 충주시,84.68,201512,"15,900",9,2002.0,4.313000e+09
5863879,충청북도 충주시,84.68,201512,"16,000",10,2002.0,4.313000e+09


In [16]:
# we check to see which addresses are not taken care of properly
# fortunately, there are only three
df_estate[df_estate[u'행자부코드'].isnull()].groupby(u'시군구').count()

,전용면적(㎡),계약년월,거래금액(만원),층,건축년도,행자부코드
시군구,,,,,,
세종특별자치시 고운동,71,71,71,71,71,0
세종특별자치시 나성동,93,93,93,93,93,0
세종특별자치시 어진동,139,139,139,139,139,0


In [18]:
# getting ready to patch things up manually
# 고운동, 나성동, 어진동 were all part of 한솔동
# moreover, 도담동, 아름동, 종촌동 were all part of 한솔동 until very recently
# so we might as well combine everything into one address
df_estate[u'시군구'].replace(to_replace=[u'세종특별자치시 고운동', u'세종특별자치시 나성동', u'세종특별자치시 어진동',
                                        u'세종특별자치시 도담동', u'세종특별자치시 아름동', u'세종특별자치시 종촌동'],
                            value=u'세종특별자치시 한솔동', inplace=True)

In [19]:
# try mapping string addresses to codes again
df_estate[u'행자부코드'] = df_estate[u'시군구'].map(sr_address_code)

In [20]:
# we see that integer values are assigned this time, so the fix was successful
df_estate.tail()

,시군구,전용면적(㎡),계약년월,거래금액(만원),층,건축년도,행자부코드
5863875,충청북도 충주시,59.76,201512,"10,500",8,1994.0,4313000000
5863876,충청북도 충주시,59.76,201512,"10,200",10,1994.0,4313000000
5863877,충청북도 충주시,84.93,201512,"19,700",12,2007.0,4313000000
5863878,충청북도 충주시,84.68,201512,"15,900",9,2002.0,4313000000
5863879,충청북도 충주시,84.68,201512,"16,000",10,2002.0,4313000000


In [21]:
# as year built is also of float type, we perform another sanity check
# again, fortunately, there are only four
df_estate[df_estate[u'건축년도'].isnull()]

,시군구,전용면적(㎡),계약년월,거래금액(만원),층,건축년도,행자부코드
234524,경기도 안양시,52.92,200606,"12,000",6,NaN,4117000000
234525,경기도 안양시,52.92,200606,"11,000",4,NaN,4117000000
356846,경기도 안양시,52.92,200609,"12,400",6,NaN,4117000000
1459878,세종특별자치시 조치원읍,66.68,200807,"5,500",4,NaN,3611025000


In [22]:
# after some investigation, apartments indexed at 234524, 234525, and 356846 can be
# reasonably assumed to be a part of the apartment complex called 태하
# we also drop a single row which cannot be identified
df_estate.loc[[234524, 234525, 356846], u'건축년도'] = 1987
df_estate.drop(1459878, inplace=True)

In [23]:
# one last sanity check
df_estate.isnull().values.any()

False

In [24]:
# convert years built to integer to save some space
df_estate[u'건축년도'] = df_estate[u'건축년도'].astype(int)

In [28]:
# further shorten address code to 5 digits
df_estate[u'행자부코드'] = df_estate[u'행자부코드'] // 100000

In [29]:
# export to a new file
df_estate.to_csv(path + 'real_estate_short.csv', index=False, encoding='utf-8')